### Imports

In [1]:
# Standard imports
import pandas as pd
import plotly.express as px
import plotly.io as pio   
pio.kaleido.scope.mathjax = None

### Load scenarios

In [2]:
df_scenario_round_1 = pd.read_csv('scenario_round_1.csv')

In [3]:
df_scenario_round_1

,Time (a.u.),Step,Energy (TeV),Emittance (μm),Intensity (1e11 ppb),Beta (m),Luminosity (1e34 cm2/s),Octupoles (A),Crabs (μrad),Chromaticity,Optics
0,0.00,Start injection,0.45,2.0,0.0,11.00,0,10,0,15,Skipped
1,0.20,End of injection,0.45,2.0,2.3,11.00,0,10,0,15,operation/optics/R2023a_A11mC11mA10mL10m_Phase...
2,0.50,End of ramp,7.00,2.0,2.3,1.00,0,450,0,15,Same_as_start_of_collapse
3,0.60,Start of collapse,7.00,2.0,2.3,1.00,0,450,0,15,round/start_collapse/opt_collapse_1000_1500_tr...
4,0.65,End of collapse,7.00,2.5,2.3,1.00,2,450,0,15,round/start_collapse/opt_collapse_1000_1500_tr...
5,0.90,Start of levelling,7.00,2.5,2.2,0.59,5,60,190,15,acc-models-lhc/strengths/round/levelling/opt_l...
6,2.00,End of levelling,7.00,2.5,1.4,0.15,5,60,190,15,acc-models-lhc/strengths/round/opt_round_150_1...
7,2.50,Dump,7.00,2.5,1.2,0.15,4,60,190,15,Same as end of levelling


### Plot scenarios

In [4]:
def plot_scenario(
    df_scenario,
    title_scenario,
    variables_to_plot=[
        "Energy (TeV)",
        "Emittance (μm)",
        "Intensity (1e11 ppb)",
        "Beta (m)",
        "Luminosity (1e34 cm2/s)",
        "Octupoles (1e2 A)",
        "Crabs (1e2 μrad)",
        "Chromaticity",
    ],
    rescale_octupoles=True,
    rescale_crabs=True,
    save_to_pdf=True,
    save_to_png=True,
):

    # Rescale variables that are too large
    if rescale_octupoles and "Octupoles (A)" in df_scenario.columns:
        df_scenario["Octupoles (A)"] = df_scenario["Octupoles (A)"] / 100
        df_scenario = df_scenario.rename(columns={"Octupoles (A)": "Octupoles (1e2 A)"})
    if rescale_crabs and "Crabs (μrad)" in df_scenario.columns:
        df_scenario["Crabs (μrad)"] = df_scenario["Crabs (μrad)"] / 100
        df_scenario = df_scenario.rename(columns={"Crabs (μrad)": "Crabs (1e2 μrad)"})

    # Do the interactive plot
    fig = px.line(
        df_scenario,
        x="Time (a.u.)",
        y=variables_to_plot,
        markers=True,
        template="simple_white",
        color_discrete_sequence=px.colors.qualitative.Dark2,
        width=1000,
        height=8 * 00,
        hover_data=["Optics"],
    )

    # Improve plot aesthetics
    fig.update_traces(
        line={"width": 1.5},
        marker=dict(size=4),
        mode="markers+lines",
        hovertemplate=None,
        hoverlabel=dict(namelength=-1),
    )
    fig.update_yaxes(showgrid=False, title="Parameter value")
    fig.update_xaxes(
        showgrid=True,
        range=[df_scenario["Time (a.u.)"].iloc[0], df_scenario["Time (a.u.)"].iloc[-1]],
        title="Time (not to scale)",
        ticktext=df_scenario["Step"],
        tickvals=df_scenario["Time (a.u.)"],
    )
    fig.update_layout(
        legend_title="Parameters",
        font_family="Latin Modern Roman",
        title_font_family="Latin Modern Roman",
        title_text=title_scenario,
        title_x=0.5,
        hovermode="x unified",
    )

    # Show and possibly save the plot
    fig.show()
    if save_to_pdf:
        fig.write_image(f"{title_scenario}.pdf")
    if save_to_png:
        fig.write_image(f"{title_scenario}.png", scale=2)

In [5]:
plot_scenario(
    df_scenario=df_scenario_round_1,
    title_scenario="HL-LHC DA scenario 1 (round)",
)